In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import math
import matplotlib as mpl
from matplotlib import rc
warnings.filterwarnings('ignore')


In [ ]:
file = np.load("data/ZFT_training_images_26k_iband_time_series_images_EPL-2.npz", allow_pickle = True)
file_full = np.load("./data/ZTF_training_images_26k_iband_image_properties_EPL-2.npz", allow_pickle = True)

In [ ]:
# array: len = 23223

z_source = file_full['z_source_list']
z_lens = file_full['z_lens_list']
theta_E = file_full['theta_E_list']
colour = file_full['colour_param']
stretch = file_full['stretch_param']
max_TD = file_full['max_time_delay']
ra_source = file_full['ra_source_list']
dec_source = file_full['dec_source_list']

In [ ]:
mag = file_full['magnification_list']

In [ ]:
# array of arrays: 23223 arrays composed of info on each image
abs_mags = file_full['app_mag_ps_list']
mag = file_full['magnification_list']

In [ ]:
catalog = file['catalog_time_series']     
labels = file['data_labels']     # array de listes de taille 2 remplies de 0 et 1      longueur : 23223
times = file['time_stamps']      # array contenant des listes de taille variable     longueur : 23223

In [ ]:
labelsbis =[]
for i in range(len(labels)):

    if (labels[i]==np.array([0, 1])).all():
        labelsbis.append(0)
    else:
        labelsbis.append(1)

In [ ]:
x = []
y = []

a = []
b = []

for indx, cat in enumerate(catalog):

    x_cat = []
    y_cat = []
    a_cat = []
    b_cat = []

    for c in cat:
        
        try:   
            x_cat.append(*c['x'])
            y_cat.append(*c['y'])

            a_cat.append(*c['a'])
            b_cat.append(*c['b'])
            
        except:
            example = 1

        #if indx==237:
            #print(c )
            #print('x', c['x'])
            
         #   print('x_cat', x_cat)
    x.append(np.array(x_cat))
    y.append(np.array(y_cat))
    a.append(np.array(a_cat))
    b.append(np.array(b_cat))

In [ ]:
for i in range(len(x)):
    
    if i==0:
        f = pd.DataFrame({'nb': i, 'label': labelsbis[i], 'x' : x[i], 'y' : y[i], 'a' : a[i], 'b' : b[i]})
        
        
    else:
        f1 = pd.DataFrame({'nb': i, 'label': labelsbis[i], 'x' : x[i], 'y' : y[i], 'a' : a[i], 'b' : b[i]})
        f = pd.concat([f, f1])

In [ ]:
nbunlensed = []
theta_E_unlensed = []
nblensed = []
theta_E_lensed = []
for i in range(len(f['label'][0].values)):
    if len(f[f['nb']==i]['x'])!=0:
        if f['label'][0].values[i]==0:
            nbunlensed.append(i)
            theta_E_unlensed.append(0)
        else:
            nblensed.append(i)
            theta_E_lensed.append(theta_E[i])

In [ ]:
lensed = f[f['label']==1]
unlensed = f[f['label']==0]

# SEARCH OF METRIC TO DISCRIMATE LENSED/UNLENSED

## USING STD FOR THE COUPLES (X,Y) AND (A,B)

In [ ]:
def stdevia(X, Y):
    if len(X)==len(Y):
        return np.sqrt((np.sum((np.mean(X)-X)**2 + (np.mean(Y)-Y)**2))/len(X))
    else:
        print('not same size')

In [ ]:
STDxlensed = []
STDalensed = []


for i in nblensed:
    X = stdevia(lensed[lensed['nb']==i]['x']-np.mean(lensed[lensed['nb']==i]['x']), lensed[lensed['nb']==i]['y']-np.mean(lensed[lensed['nb']==i]['y']))
    A = stdevia(lensed[lensed['nb']==i]['a']-np.mean(lensed[lensed['nb']==i]['a']), lensed[lensed['nb']==i]['b']-np.mean(lensed[lensed['nb']==i]['b']))
    if X!= 0 and len(lensed[lensed['nb']==i]['x'])>1:
        STDxlensed.append(X)
        STDalensed.append(A)

In [ ]:
STDxunlensed = []
STDaunlensed = []

for i in nbunlensed:
    X = stdevia(unlensed[unlensed['nb']==i]['x']-np.mean(unlensed[unlensed['nb']==i]['x']), unlensed[unlensed['nb']==i]['y']-np.mean(unlensed[unlensed['nb']==i]['y']))
    A = stdevia(unlensed[unlensed['nb']==i]['a']-np.mean(unlensed[unlensed['nb']==i]['a']), unlensed[unlensed['nb']==i]['b']-np.mean(unlensed[unlensed['nb']==i]['b']))
    if X!=0 and len(unlensed[unlensed['nb']==i]['x'])>1:
        STDxunlensed.append(X)
        STDaunlensed.append(A)

In [ ]:
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)

plt.figure(figsize=(6,6))
plt.plot(STDalensed, STDxlensed, 'r.', label='lensed')
plt.plot(STDaunlensed, STDxunlensed, 'b.', label = 'unlensed')
plt.xlabel('metric ab')
plt.ylabel('metric xy')
plt.title('std deviation (x,y) and (a,b)');
plt.legend();

## USING RELATIVE STD DEVIATION $\sigma / \bar{x}$

In [ ]:
def relastdevia(X, Y):
    if len(X)==len(Y):
        return np.sqrt((np.sum(((np.mean(X)-X)/np.mean(X))**2 + (((np.mean(Y)-Y))/np.mean(Y))**2))/len(X))
    else:
        print('not same size')

In [ ]:
RSTDxlensed = []
RSTDalensed = []

for i in nblensed:
    X = relastdevia(lensed[lensed['nb']==i]['x'], lensed[lensed['nb']==i]['y'])
    A = relastdevia(lensed[lensed['nb']==i]['a'], lensed[lensed['nb']==i]['b'])
    if x !=0:
        RSTDxlensed.append(X)
        RSTDalensed.append(A)

In [ ]:
RSTDxunlensed = []
RSTDaunlensed = []

for i in nbunlensed:
    X = relastdevia(unlensed[unlensed['nb']==i]['x'], unlensed[unlensed['nb']==i]['y'])
    A = relastdevia(unlensed[unlensed['nb']==i]['a'], unlensed[unlensed['nb']==i]['b'])
    if x!=0:
        RSTDxunlensed.append(X)
        RSTDaunlensed.append(A)

In [ ]:
plt.figure(figsize=(6,6))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)

plt.plot(RSTDalensed, RSTDxlensed, 'r.', label='lensed')
plt.plot(RSTDaunlensed, RSTDxunlensed, 'b.', label = 'unlensed')
plt.xlabel('metric ab')
plt.ylabel('metric xy')
plt.title('relative standard deviation (x,y) and (a,b)');
plt.legend();

## USING LENGHT MAX FOR (X,Y)/ (A,B) AND CENTER (0,0)

In [ ]:
from scipy.spatial import distance

distxlensed = []
distalensed = []
C = (0,0)

for i in nblensed:
    Dx = []
    Da = []
    X = lensed[lensed['nb']==i]['x'].copy()
    Y = lensed[lensed['nb']==i]['y'].copy()
    MX = np.mean(X)
    MY = np.mean(Y)
    X = X - MX
    Y = Y - MY
    A = lensed[lensed['nb']==i]['a'].copy()
    B = lensed[lensed['nb']==i]['b'].copy()
    MA = np.mean(A)
    MB = np.mean(B)
    A = A - MA
    B = B - MB
    if len(X)!=0:
        for j in range(len(X)):
            Dx.append(distance.euclidean(C, (X[j], Y[j])))
        distxlensed.append(max(Dx))
        for j in range(len(A)):
            Da.append(distance.euclidean(C, (A[j], B[j])))
        distalensed.append(min(Da))

In [ ]:
distxunlensed = []
distaunlensed = []

for i in nbunlensed:
    Dx = []
    Da = []
    X = unlensed[unlensed['nb']==i]['x']
    Y = unlensed[unlensed['nb']==i]['y']
    MX = np.mean(X)
    MY = np.mean(Y)
    X = X - MX
    Y = Y - MY
    A = unlensed[unlensed['nb']==i]['a']
    B = unlensed[unlensed['nb']==i]['b']
    MA = np.mean(A)
    MB = np.mean(B)
    A = A - MA
    B = B - MB
    if len(X)!=0:
        for j in range(len(X)):
            Dx.append(distance.euclidean(C, (X[j], Y[j])))
        distxunlensed.append(max(Dx))
        for j in range(len(A)):
            Da.append(distance.euclidean(C, (A[j], B[j])))
        distaunlensed.append(min(Da))

In [ ]:
plt.figure(figsize=(6,6))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)
plt.plot(distalensed, distxlensed, 'r.', label = 'lensed')
plt.plot(distaunlensed, distxunlensed, 'b.', label = 'unlensed')
plt.xlabel('metric ab')
plt.ylabel('metric xy')
plt.legend()
plt.title('max distance(x,y) and max distance (a,b) and the center');

## USING LENGHT MAX FOR (X,Y)/ MINIMUM (A,B) BETWEEN EACH POINT

In [ ]:
def distances(L1 , L2):           # to obtain distances once from positions lists L1 (for x) and L2 (for y for ex)
    D = []
    l1 = L1.copy()
    l2 = L2.copy()
    while len(l1)>1:
        for j in range(1, len(l1)):
            D.append(distance.euclidean((l1[0],l2[0]), (l1[j], l2[j])))
            #print(l1[0], l2[0], l1[j], l2[j])
        l1 = l1[1:]
        l2 = l2[1:] 
    return D 

In [ ]:
Maxxlensed = []
Minalensed = []

for i in nblensed:
    if len(lensed[lensed['nb']==i]['x'])>1:
        X = lensed[lensed['nb']==i]['x'].copy()
        X = X - np.mean(X)
        Y = lensed[lensed['nb']==i]['y'].copy()
        Y = Y - np.mean(Y)
        A = lensed[lensed['nb']==i]['a'].copy()
        A = A - np.mean(A)
        B = lensed[lensed['nb']==i]['b'].copy()
        B = B-np.mean(B)
        Maxxlensed.append(max(distances(list(X), list(Y))))
        Minalensed.append(min(distances(list(A), list(B))))

In [ ]:
Maxxunlensed = []
Minaunlensed = []

for i in nbunlensed:

    #print(len(unlensed[unlensed['nb']==i]['x']))
    if len(unlensed[unlensed['nb']==i]['x'])>1:
        #print('here')
        X = unlensed[unlensed['nb']==i]['x'].copy()
        X = X - np.mean(X)
        Y = unlensed[unlensed['nb']==i]['y'].copy()
        Y = Y - np.mean(Y)
        A = unlensed[unlensed['nb']==i]['a'].copy()
        A = A - np.mean(A)
        B = unlensed[unlensed['nb']==i]['b'].copy()
        B = B-np.mean(B)
        Maxxunlensed.append(max(distances(list(X), list(Y))))
        Minaunlensed.append(min(distances(list(A), list(B))))

In [ ]:
plt.figure(figsize=(6,6))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)
plt.plot(Minalensed, Maxxlensed, 'r.', label = 'lensed')
plt.plot(Minaunlensed, Maxxunlensed, 'b.', label = 'unlensed')
plt.xlabel('metric ab')
plt.ylabel('metric xy')
plt.legend()
plt.title('max distance(x,y) and min distance(a,b)');

In [ ]:
########################### WITH OUTLIERS ###############################
Maxxoutliers = []

for i in range(len(STDxoutliers)):
    X = STDxoutliers[i].values
    X = X - np.mean(X)
    Y = STDyoutliers[i].values
    Y = Y - np.mean(Y)
    Maxxoutliers.append(max(distances(list(X), list(Y))))

## USING LENGHT MAX FOR (X,Y) BETWEEN EACH POINT AND MIN(A- B) 

In [ ]:
def subst(L1, L2):
    return([abs(L1[i]-L2[i]) for i in range(len(L1))])

In [ ]:
Diffalensed = []
for i in nblensed:
    if len(lensed[lensed['nb']==i]['a'].copy())>1:
        Diffalensed.append(min(subst(lensed[lensed['nb']==i]['a'].copy(), lensed[lensed['nb']==i]['b'].copy())))

In [ ]:
Diffaunlensed = []
for i in nbunlensed:
    if len(unlensed[unlensed['nb']==i]['a'].copy())>1:
        Diffaunlensed.append(min(subst(unlensed[unlensed['nb']==i]['a'].copy(), unlensed[unlensed['nb']==i]['b'].copy())))

In [ ]:
Mindiffaoutliers = []

for i in range(len(STDaoutliers)):
    Mindiffaoutliers.append(min(subst(STDaoutliers[i].values, STDboutliers[i].values)))

In [ ]:
plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)
plt.plot(Diffalensed, Maxxlensed, 'r.', label = 'lensed')
plt.plot(Diffaunlensed, Maxxunlensed, 'b.', label = 'unlensed')
plt.plot(Mindiffaoutliers, Maxxoutliers, 'g+', label='overlapping')
plt.xlabel('metric ab')
plt.ylabel('metric xy')
plt.legend()
plt.title('max distance(x,y) and min(a-b)');

## USING LENGHT MAX FOR (X,Y) BETWEEN EACH POINT AND MEAN(A- B) 

In [ ]:
Meandiffalensed = []
for i in nblensed:
    if len(lensed[lensed['nb']==i]['a'].copy())>1:
        Meandiffalensed.append(np.mean(subst(lensed[lensed['nb']==i]['a'].copy(), lensed[lensed['nb']==i]['b'].copy())))

In [ ]:
Meandiffaunlensed = []
for i in nbunlensed:
    if len(unlensed[unlensed['nb']==i]['a'].copy())>1:
        Meandiffaunlensed.append(np.mean(subst(unlensed[unlensed['nb']==i]['a'].copy(), unlensed[unlensed['nb']==i]['b'].copy())))

In [ ]:
Meandiffaoutliers = []

for i in range(len(STDaoutliers)):
    Meandiffaoutliers.append(np.mean(subst(STDaoutliers[i].values, STDboutliers[i].values)))

In [ ]:
plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)
plt.plot(Meandiffalensed, Maxxlensed, 'r.', label = 'lensed')
plt.plot(Meandiffaunlensed, Maxxunlensed, 'b.', label = 'unlensed')
#plt.plot(Meandiffaoutliers, Maxxoutliers, 'g+', label ='overlapping')
plt.xlabel('metric ab')
plt.ylabel('metric xy')
plt.legend()
plt.title('max distance(x,y) and mean(a-b)');

## USING STD FOR (X,Y) BETWEEN EACH POINT AND MEAN(NORM(A-B))

In [ ]:
def substnorm(L1, L2):
    return([abs(L1[i]-L2[i])/L1[i] for i in range(len(L1))])

In [ ]:
Meandiffnormalensed = []
for i in nblensed:
    if len(lensed[lensed['nb']==i]['a'].copy())>1:
        Meandiffnormalensed.append(np.mean(substnorm(lensed[lensed['nb']==i]['a'].copy(), lensed[lensed['nb']==i]['b'].copy())))

In [ ]:
Meandiffnormaunlensed = []
for i in nbunlensed:
    if len(unlensed[unlensed['nb']==i]['a'].copy())>1:
        Meandiffnormaunlensed.append(np.mean(substnorm(unlensed[unlensed['nb']==i]['a'].copy(), unlensed[unlensed['nb']==i]['b'].copy())))

In [ ]:
plt.figure(figsize=(8,8))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)
plt.plot(Meandiffnormalensed, STDxlensed, 'r.', label = 'lensed')
plt.plot(Meandiffnormaunlensed, STDxunlensed, 'b.', label = 'unlensed')
plt.xlabel('Mean of differences (a,b)')
plt.ylabel('Standard deviation (x,y)')
plt.legend()
plt.ylim(0, 0.5)
plt.title('Metric (x,y) as a function of the metric (a,b)')
plt.show()

## USING STD DEVIATION FOR (X,Y) AND MEDIAN(NORM(A-B))

In [ ]:
Mediandiffnormalensed = []
for i in nblensed:
    if len(lensed[lensed['nb']==i]['a'].copy())>1:
        Mediandiffnormalensed.append(np.median(substnorm(lensed[lensed['nb']==i]['a'].copy(), lensed[lensed['nb']==i]['b'].copy())))

In [ ]:
Mediandiffnormaunlensed = []
for i in nbunlensed:
    if len(unlensed[unlensed['nb']==i]['a'].copy())>1:
        Mediandiffnormaunlensed.append(np.median(substnorm(unlensed[unlensed['nb']==i]['a'].copy(), unlensed[unlensed['nb']==i]['b'].copy())))

In [ ]:
plt.figure(figsize=(8,8))
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amssymb}')
plt.rc('font', family='serif')
plt.rc('font', size=15)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)
plt.plot(Mediandiffnormalensed, STDxlensed, 'r.', label = 'lensed')
plt.plot(Mediandiffnormaunlensed, STDxunlensed, 'b.', label = 'unlensed')
plt.xlabel('Median of difference (a,b)')
plt.ylabel('Standard deviation (x,y)')
plt.legend()
plt.title('Metric (x,y) as a function of the metric (a,b)')
plt.show()